In [2]:
import torch
import torch.nn.functional as F
import torch_geometric as tg
from torch_geometric.nn import knn_graph, fps, knn
from torch_geometric.data import Data
import torch_geometric.io as tgio
from torch_geometric.io import read_off, read_ply
from torch_geometric.transforms import SamplePoints, NormalizeScale
import os, sys, time, random
import numpy as np
import pptk


In [2]:
def process_ply(ply_path: str, n_patch: int = 100):
    r"""
    Processes PLY file from path, convert to PC
    """
    mesh_data = read_ply(ply_path)
    print(mesh_data)
    # pc = SamplePoints(num=2048 * n_patch)(mesh_data).pos
    pc = mesh_data.pos
    n_pts = pc.shape[0]
    # select N kernels: [N, ]
    patch_kernel = fps(pc, ratio=n_patch / n_pts + 1e-6)
    # select N patches: [N, 2048]
    patches = knn(pc[patch_kernel], pc, k=2048, num_workers=16)
    return pc[patches] # [N, 2048, F]

In [20]:
dataset_path = '/data/pkurei/mpeg/dataset'
ply=os.path.join(dataset_path, 'longdress', 'Ply', 'longdress_vox10_1099.ply')
ply

'/data/pkurei/mpeg/dataset/longdress/Ply/longdress_vox10_1099.ply'

In [4]:
res=process_ply(ply_path=ply)

Data(face=[0], pos=[765821, 3])


In [6]:
res.shape

torch.Size([2, 77347921, 3])

In [17]:
import openmesh
from openmesh import *
# option=Options()
# option += openmesh.Options.vertexColor
mesh = openmesh.TriMesh()
mesh.request_vertex_colors()
mesh = openmesh.read_trimesh(ply)
mesh.points()

AttributeError: 'openmesh.TriMesh' object has no attribute 'colors'

In [21]:
for vh in mesh.vertices():
    if(vh.idx() % 10000 == 0):
        print(vh.idx())
        print(mesh.color(vh))
        

0
[1.4784527e+00 3.0907039e-41 3.0325372e-15 4.5765007e-41]
10000
[0.        3.171875  0.        3.6757812]
20000
[0.        3.3515625 0.        3.3789062]
30000
[0.        3.7226562 0.        3.1328125]
40000
[0.        3.6777344 0.        3.7050781]
50000
[0.        3.5546875 0.        3.0703125]
60000
[0.        3.7226562 0.        3.5097656]
70000
[0.        2.921875  0.        3.7089844]
80000
[0.        3.6972656 0.        3.3867188]
90000
[0.        3.6855469 0.        3.5      ]
100000
[0.        3.7304688 0.        3.6835938]
110000
[0.        3.3828125 0.        3.7509766]
120000
[0.        3.4960938 0.        3.9707031]
130000
[0.        3.234375  0.        3.6914062]
140000
[0.        3.7910156 0.        3.4960938]
150000
[0.        3.6835938 0.        3.8134766]
160000
[0.        3.4726562 0.        3.6875   ]
170000
[0.        3.7783203 0.        3.6367188]
180000
[0.        3.6914062 0.        3.8632812]
190000
[0.        3.6894531 0.        3.6992188]
200000
[0.        

In [34]:
mesh.request_vertex_colors()
mesh.update_colors()

AttributeError: 'openmesh.TriMesh' object has no attribute 'update_colors'

In [13]:
dir(openmesh)

['BaseHandle',
 'EdgeHandle',
 'EdgeIter',
 'FaceEdgeIter',
 'FaceFaceIter',
 'FaceHalfedgeIter',
 'FaceHandle',
 'FaceIter',
 'FaceVertexIter',
 'HalfedgeHandle',
 'HalfedgeIter',
 'HalfedgeLoopIter',
 'PolyMesh',
 'PolyMeshDecimater',
 'PolyMeshModAspectRatio',
 'PolyMeshModAspectRatioHandle',
 'PolyMeshModBase',
 'PolyMeshModEdgeLength',
 'PolyMeshModEdgeLengthHandle',
 'PolyMeshModHausdorff',
 'PolyMeshModHausdorffHandle',
 'PolyMeshModIndependentSets',
 'PolyMeshModIndependentSetsHandle',
 'PolyMeshModNormalDeviation',
 'PolyMeshModNormalDeviationHandle',
 'PolyMeshModNormalFlipping',
 'PolyMeshModNormalFlippingHandle',
 'PolyMeshModProgMesh',
 'PolyMeshModProgMeshHandle',
 'PolyMeshModProgMeshInfo',
 'PolyMeshModQuadric',
 'PolyMeshModQuadricHandle',
 'PolyMeshModRoundness',
 'PolyMeshModRoundnessHandle',
 'TriMesh',
 'TriMeshDecimater',
 'TriMeshModAspectRatio',
 'TriMeshModAspectRatioHandle',
 'TriMeshModBase',
 'TriMeshModEdgeLength',
 'TriMeshModEdgeLengthHandle',
 'TriMeshMo

In [22]:
from plyfile import PlyData, PlyElement

In [24]:
mesh=PlyData.read(ply)

In [53]:
mesh.elements[0].data['red'].shape

(824064,)

In [41]:
mesh.elements[0]

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar')), count=824064, comments=[])

In [45]:
mesh.elements[0].data

array([(231.,  14., 411., 124,  85,  64),
       (231.,  15., 411., 127,  86,  65),
       (231.,  13., 412., 118,  83,  63), ...,
       (320., 785., 513., 163, 150, 144),
       (320., 786., 512., 156, 146, 143),
       (320., 787., 512., 139, 130, 126)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [48]:
a=torch.randn([1000,3])

In [50]:
a.to(torch.long).dtype

torch.int64

In [52]:
torch.float32

torch.float32

In [55]:
824064/2048

402.375

In [63]:
a=torch.from_numpy(mesh.elements[0].data['x'].copy())

In [64]:
a.shape

torch.Size([824064])

In [71]:
x=torch.randn([1000,3])

In [72]:
y=x[torch.randperm(len(x))[:10]]

In [74]:
y.shape

torch.Size([10, 3])

In [75]:
from torch_geometric.nn import knn

In [87]:
res=knn(x,y,k=11)
res.shape

torch.Size([2, 110])

In [86]:
torch.einsum('ijk->jki',res.reshape(2, 10, 11)).shape

torch.Size([10, 11, 2])

In [92]:
res[1].reshape([10,11])

tensor([[758, 447, 618, 662,  32, 311,  68, 633, 191, 403, 982],
        [257,  42, 592, 551, 765, 400, 933, 577, 568, 831, 176],
        [314, 266, 696, 452, 936, 464, 462, 676, 938, 587, 824],
        [423, 399,  78, 893, 601, 918, 748, 586, 486, 396, 267],
        [455, 581, 491, 126, 804, 177, 239, 494, 657, 782, 829],
        [260, 990, 331, 282, 912, 322, 989, 553, 469, 431, 156],
        [577, 592, 400,  60, 831, 701,  38, 933, 629,   8, 176],
        [648, 185, 759, 913, 181, 379, 868, 928, 799, 643, 355],
        [467, 548, 453,  52, 143, 680, 536, 876, 398,  15, 159],
        [180, 621, 461, 601, 937, 393, 483, 348, 739, 837, 893]])